In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('ks.csv')

In [3]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Состояние,Инвесторов,Страна,Собрано в долларах,Цель в долларах
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,successful,23,US,600.00,600.00
1,Arcade County (Canceled),Games,Games,USD,2012-04-29,2012-03-30 23:40:45,canceled,5,US,71.00,9000.00
2,Hayashi Skate Co. Solar Skateboard backpack,Accessories,Fashion,CAD,2017-07-22,2017-05-23 23:00:13,canceled,8,CA,360.36,2391.77
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,failed,20,US,502.00,10000.00
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,successful,62,US,2414.00,1400.00


In [4]:
data.shape

(378661, 11)

In [5]:
### Оставим только нужные состояния

data = data[data['Состояние'].isin(['failed', 'successful'])]

### Что берем за таргет? 2 варианта:

- Будем решать задачу классификации, разметив объекты следующим образом: те проекты, у которых успешный статус, единичкой, а остальные ноликами

- Можем предсказывать просто собранное количество денег, применять модель, а потом уже смотреть, нужная ли сумма получилось. Тогда мы решаем задачу регрессии.

In [6]:
### Создадим колонку таргет1 (бинарный таргет)

data.loc[(data['Состояние'] == 'failed'), 'таргет1'] = 0
data['таргет1'] = data['таргет1'].fillna(1)

In [7]:
data = data.drop('Состояние', axis=1)

In [8]:
### Создадим вещественную колонку таргет2
data = data.rename({'Собрано в долларах':'таргет2'}, axis=1)
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Инвесторов,Страна,таргет2,Цель в долларах,таргет1
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,23,US,600.00,600.0,1.0
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,20,US,502.00,10000.0,0.0
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,62,US,2414.00,1400.0,1.0
5,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03 16:17:21,86,US,10030.88,10000.0,1.0
6,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12 05:19:53,0,US,0.00,10000.0,0.0


### Чтобы получить матрицу объектов, зачастую нужно обработать сырые данные, то есть извлечь из имеющихся таблиц признаки там, где они не даны явно

In [9]:
### Оставим от колонок с датами разницу в днях 

data['Дедлайн'] = pd.to_datetime(data['Дедлайн'])
data['Дата публикации'] = pd.to_datetime(data['Дата публикации'])

In [10]:
data['Срок'] = (data['Дедлайн'] - data['Дата публикации']).dt.days

In [11]:
### Выделим год запуска проекта
data['Год публикации'] = data['Дата публикации'].dt.year

In [12]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Инвесторов,Страна,таргет2,Цель в долларах,таргет1,Срок,Год публикации
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,23,US,600.00,600.0,1.0,31,2012
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,20,US,502.00,10000.0,0.0,29,2016
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,62,US,2414.00,1400.0,1.0,29,2015
5,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03 16:17:21,86,US,10030.88,10000.0,1.0,44,2013
6,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12 05:19:53,0,US,0.00,10000.0,0.0,29,2012


### Иногда может казаться, что признаков мало. Стоит подумать над тем, что еще может объяснять таргет-переменную.

In [13]:
Macro = pd.read_excel("macrofeatures.xlsx", engine="openpyxl")

Macro.head()

FileNotFoundError: [Errno 2] No such file or directory: 'macrofeatures.xlsx'

In [ ]:
### Смерджим колонки. Оставим от макрофичей какую-нибудь одну.

Macro = Macro[['Close_brent', 'dlk_cob_date']].drop_duplicates()

In [ ]:
Macro['dlk_cob_date'] = pd.to_datetime(Macro['dlk_cob_date'])

In [ ]:
data['Дата публикации'] = data['Дата публикации'].astype('datetime64[ns]')

In [ ]:
data = pd.merge(data,
         Macro,
         left_on=['Дата публикации'],
         right_on=['dlk_cob_date'],
         how='left')

In [ ]:
data = data.sort_values('Дата публикации')

In [ ]:
data['Close_brent'] = data['Close_brent'].fillna(34.41)

In [ ]:
data = data.drop(['Дедлайн', 'Дата публикации', 'dlk_cob_date'], axis=1)

In [ ]:
data.head()

### Финально, стоит избавиться от колонок, которые мы не считаем признаками

In [ ]:
### Уберем колонки Дедлайн, Дата запуска, Инвесторы, Название, Страна

data = data.drop(['Название', 'Страна', 'Инвесторов'], axis=1)

In [ ]:
data.head()

## One-hot encoding

In [ ]:
Image('ONEHOT.png')

In [ ]:
### One-hot encoding колонки с валютой

data = pd.concat((data, pd.get_dummies(data['Валюта'])), axis=1)
data = data.drop(['Валюта'], axis=1)

In [ ]:
data.head()

In [ ]:
### Почистим данные от лишней информации 
data = data.drop('AUD', axis=1)

In [ ]:
### Аналогично закодируем Главную категорию
data = pd.concat((data, pd.get_dummies(data['Главная категория'])), axis=1)
data = data.drop(['Главная категория'], axis=1)

In [ ]:
data.head()

In [ ]:
data = data.drop('Games', axis=1)

In [ ]:
### А что с колонкой Категория?

len(data['Категория'].unique())

## Mean-target encoding (Счетчики)

In [ ]:
Image('Счетчики.png')

In [14]:
### Произведем mean-target-encoding колонки Категория

data['Категория'] = data['Категория'].map(data.groupby(['Категория'])['таргет2'].mean())

In [16]:
data.groupby('Категория')['таргет2'].mean()

Категория
317.617947        317.617947
631.714603        631.714603
946.153400        946.153400
953.058936        953.058936
1012.834051      1012.834051
                    ...     
57306.019691    57306.019691
65854.270745    65854.270745
73083.996370    73083.996370
76876.211356    76876.211356
95287.899486    95287.899486
Name: таргет2, Length: 159, dtype: float64

In [76]:
data.head()

,Категория,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,625.0,1000.0,0.0,39,2009,34.41,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241929,3591.033473,22.0,80000.0,0.0,87,2009,34.41,0,0,0,...,0,0,1,0,0,0,0,0,0,0
244460,3661.424550,35.0,20.0,1.0,8,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,4321.245721,145.0,99.0,1.0,79,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,6035.989239,387.0,1900.0,0.0,28,2009,34.41,0,0,0,...,0,1,0,0,0,0,0,0,0,0


## Определимся с таргетом

In [77]:
data = data.drop('таргет1', axis=1)

In [78]:
### Разделим данные на объекты и ответы

X = data.drop('таргет2', axis=1)
Y = data['таргет2']

In [80]:
Y

176128    625.00
241929     22.00
244460     35.00
80845     145.00
181197    387.00
           ...  
247730    269.84
318187    544.77
264474    202.00
293634    265.00
23315     513.60
Name: таргет2, Length: 331675, dtype: float64

### sklearn

In [90]:
### Сейчас будет магия!
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, Y)

X['Предсказание'] = model.predict(X)

X.head()

,Категория,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,GBP,...,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater,Предсказание
176128,6035.989239,1000.0,39,2009,34.41,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3462.151236
241929,3591.033473,80000.0,87,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5388.839779
244460,3661.424550,20.0,8,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1332.724207
80845,4321.245721,99.0,79,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5271.256319
181197,6035.989239,1900.0,28,2009,34.41,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2529.810713


In [91]:
data.head()

,Категория,таргет2,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,...,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater,prediction
176128,6035.989239,625.0,1000.0,39,2009,34.41,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3462.151236
241929,3591.033473,22.0,80000.0,87,2009,34.41,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5388.839779
244460,3661.424550,35.0,20.0,8,2009,34.41,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1332.724207
80845,4321.245721,145.0,99.0,79,2009,34.41,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5271.256319
181197,6035.989239,387.0,1900.0,28,2009,34.41,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2529.810713


In [50]:
#Запишем результат в data.csv
